In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os


In [3]:
sns.set(style='darkgrid')


In [4]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sample = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')


In [5]:
train['sex'].fillna('unknown', inplace=True)
test['sex'].fillna('unknown', inplace=True)

train['age_approx'].fillna(train['age_approx'].mode().values[0], inplace=True)
test['age_approx'].fillna(test['age_approx'].mode().values[0], inplace=True)

train['anatom_site_general_challenge'].fillna('unknown', inplace=True)
test['anatom_site_general_challenge'].fillna('unknown', inplace=True)


/tmp/ipykernel_8/1992711877.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['sex'].fillna('unknown', inplace=True)
/tmp/ipykernel_8/1992711877.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

In [6]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

train['sex_enc'] = enc.fit_transform(train.sex.astype('str'))
test['sex_enc'] = enc.transform(test.sex.astype('str'))

train['age_enc'] = enc.fit_transform(train.age_approx.astype('str'))
test['age_enc'] = enc.transform(test.age_approx.astype('str'))

train['anatom_enc'] = enc.fit_transform(train.anatom_site_general_challenge.astype('str'))
test['anatom_enc'] = enc.transform(test.anatom_site_general_challenge.astype('str'))


In [7]:
train.head(5)


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,sex_enc,age_enc,anatom_enc
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,1,8,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,0,8,6
2,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,0,8,0
3,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,0,10,6
4,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0,0,7,1


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import roc_auc_score


In [9]:
train.columns


Index(['image_name', 'patient_id', 'sex', 'age_approx',
       'anatom_site_general_challenge', 'diagnosis', 'benign_malignant',
       'target', 'sex_enc', 'age_enc', 'anatom_enc'],
      dtype='object')

In [10]:
features = [
            'sex_enc',
            'age_enc',
            'anatom_enc'
]


In [11]:
X = train[features]
y = train['target']

x_test = test[features]


In [12]:
model = XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
                    importance_type='gain', interaction_constraints=None,
                     min_child_weight=1, missing=None, monotone_constraints=None,
                     n_estimators=700, n_jobs=-1, nthread=-1, num_parallel_tree=1
                    )

kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(model, X, y, cv=kfold, scoring='roc_auc', verbose=3)
cv_results.mean()


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 370, in _score
    y_pred = method_caller(clf, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py", line 1168, in predict
    predts = self.get_booster().inpla

[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 370, in _score
    y_pred = method_caller(clf, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py", line 1168, in predict
    predts = self.get_booster().inpla

[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 370, in _score
    y_pred = method_caller(clf, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py", line 1168, in predict
    predts = self.get_booster().inpla

[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s
[CV] END .................................. score: (test=nan) total time=   0.1s


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 370, in _score
    y_pred = method_caller(clf, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py", line 1168, in predict
    predts = self.get_booster().inpla

nan

In [13]:
xgb =XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
                    importance_type='gain', interaction_constraints=None,
                     min_child_weight=1, missing=None, monotone_constraints=None,
                     n_estimators=700, n_jobs=-1, nthread=-1, num_parallel_tree=1
                    )
xgb.fit(X, y)
pred = xgb.predict(x_test)


XGBoostError: [03:19:27] /workspace/include/xgboost/json.h:630: Invalid type for: `missing`, expecting one of the: {``Number`, `Integer`}, got: `Null`
Stack trace:
  [bt] (0) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x15c2ca) [0x7f8f3d83a2ca]
  [bt] (1) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x194354) [0x7f8f3d872354]
  [bt] (2) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x1944b8) [0x7f8f3d8724b8]
  [bt] (3) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x16d5d8) [0x7f8f3d84b5d8]
  [bt] (4) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(XGBoosterPredictFromDense+0x2ab) [0x7f8f3d84bdcb]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7f8fad349e2e]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7f8fad346493]
  [bt] (7) /usr/lib/python3.11/lib-dynload/_ctypes.cpython-311-x86_64-linux-gnu.so(+0xa4d8) [0x7f8fadd114d8]
  [bt] (8) /usr/lib/python3.11/lib-dyn

In [14]:
sub = pd.DataFrame({'image_name':test.image_name.values,
                    'target':pred})
sub.to_csv('submission.csv',index = False)


NameError: name 'pred' is not defined

In [15]:
sub.head()


NameError: name 'sub' is not defined